In [ ]:
pip install requests

In [ ]:
pip install lxml

In [ ]:
from bottle import Bottle,run,template,request,redirect

app=Bottle()

@app.route('/')
@app.route('/games')
def game():
    class Game:

        def __init__(self,title,price,tags,platforms):
                self.title= title
                self.price=  price
                self.tags= tags
                self.platforms= platforms

    import requests
    import lxml.html

    html = requests.get ('https://store.steampowered.com/explore/new/')

    #print(html.content)

    doc = lxml.html.fromstring(html.content)

    newReleases = doc.xpath('//div[@id="tab_newreleases_content"]')[0]

    #print(newReleases)

    tittles=newReleases.xpath('.//div[@class="tab_item_name"]/text()')

    #print(tittles)

    prices = newReleases.xpath('.//div[@class="discount_final_price"]/text()')

    #print(prices)

    tags = newReleases.xpath('.//div[@class="tab_item_top_tags"]')

    totalTags=[]

    for tag in tags:
        totalTags.append(tag.text_content())

    totalTags = [tag.split(', ') for tag in totalTags]

    #print(totalTags)

    platformsDiv = newReleases.xpath('.//div[@class="tab_item_details"]')

    totalPlatforms=[]

    for game in platformsDiv:
        namePlatform = game.xpath('.//span[contains(@class, "platform_img")]')
        platforms=[t.get('class').split(' ')[-1] for t in namePlatform]
        if 'had_separator' in platforms:
            platforms.remove('had separator')
        totalPlatforms.append(platforms)

    
    #print(totalPlatforms)
    listGames = []

    for info in zip(tittles,prices,totalTags,totalPlatforms):
        lista=[info[0],info[1],info[2],info[3]]
        listGames.append(lista)
    
    output=template('table_games',rows=listGames)
    return output
run(app,host='0.0.0.0',port=5000,debug=True)